sift关键点检测与匹配

In [2]:
import cv2
import os

def sift(filename):
    img = cv2.imread(filename) # 读取文件
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转化为灰度图
    sift = cv2.SIFT.create()
    keyPoint, descriptor = sift.detectAndCompute(img, None)  # 特征提取得到关键点以及对应的描述符（特征向量）
    return img, keyPoint, descriptor


def surf(filename):
    img = cv2.imread(filename)  # 读取文件
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转化为灰度图
    sift = cv2.xfeatures2d_SURF.create()
    keyPoint, descriptor = sift.detectAndCompute(img, None)  # 特征提取得到关键点以及对应的描述符（特征向量）
    return img, keyPoint, descriptor


def orb(filename):
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)  # 读取文件
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转化为灰度图
    orb = cv2.ORB.create()
    keyPoint, descriptor = orb.detectAndCompute(img, None)  # 特征提取得到关键点以及对应的描述符（特征向量）
    return img, keyPoint, descriptor



def match(filename1, filename2):
    img1, kp1, des1 = sift(filename1)
    img2, kp2, des2 = sift(filename2)
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)  # sift的normType应该使用NORM_L2或者NORM_L1

    # img1, kp1, des1 = orb(filename1)
    # img2, kp2, des2 = orb(filename2)
    # bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)  # orb的normType应该使用NORM_HAMMING
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    # for m in matches:
    #     print(m.distance)
    knnMatches = bf.knnMatch(des1, des2, k=1)  # drawMatchesKnn

    # 过滤
    for m in matches:
        for n in matches:
            if (m != n and m.distance >= n.distance * 0.75):
                matches.remove(m)
                break

    # mean_distance = sum(match.distance for match in matches) / len(matches)
    # similarity = 1 / mean_distance  # 距离越小，相似度越高
    # print(similarity)

    img = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], img2, flags=2)
    cv2.imshow("matches", img)
    cv2.waitKey(0)


imageDir='images'
for item in os.listdir(imageDir):
    item_path = os.path.join(imageDir, item)
    if os.path.isdir(item_path):
        match(os.path.join(item_path,'1.jpg'), os.path.join(item_path,'2.jpg'))
